In [1]:
import pandas as pd

In [45]:
df = pd.read_csv("train.csv").drop(["PassengerId", "Ticket", "Cabin"], axis=1)

In [51]:
df["is_mrs"] = df["Name"].apply(lambda x: 1 if "Mrs" in x else 0)

In [52]:
df["is_mrs"].value_counts()

0    762
1    129
Name: is_mrs, dtype: int64

In [20]:
df["Embarked"].isna().sum()

2

In [21]:
df[df["Embarked"].isna()]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
61,1,1,female,38.0,0,0,80.0,NaN
829,1,1,female,62.0,0,0,80.0,NaN


In [53]:
df = df.dropna(subset=["Embarked"])

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Name      889 non-null    object 
 3   Sex       889 non-null    object 
 4   Age       712 non-null    float64
 5   SibSp     889 non-null    int64  
 6   Parch     889 non-null    int64  
 7   Fare      889 non-null    float64
 8   Embarked  889 non-null    object 
 9   is_mr     889 non-null    int64  
 10  is_mrs    889 non-null    int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 83.3+ KB


In [55]:
X = df.drop(["Survived"], axis=1)
y = df["Survived"]

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [57]:
X_train["Embarked"].value_counts()

S    432
C    110
Q     53
Name: Embarked, dtype: int64

In [58]:
X_train

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,is_mr,is_mrs
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,51.8625,S,0,0
576,2,"Garside, Miss. Ethel",female,34.0,0,0,13.0000,S,0,0
445,1,"Dodge, Master. Washington",male,4.0,0,2,81.8583,S,0,0
74,3,"Bing, Mr. Lee",male,32.0,0,0,56.4958,S,0,0
670,2,"Brown, Mrs. Thomas William Solomon (Elizabeth ...",female,40.0,1,1,39.0000,S,1,1
...,...,...,...,...,...,...,...,...,...,...
107,3,"Moss, Mr. Albert Johan",male,NaN,0,0,7.7750,S,0,0
271,3,"Tornquist, Mr. William Henry",male,25.0,0,0,0.0000,S,0,0
862,1,"Swift, Mrs. Frederick Joel (Margaret Welles Ba...",female,48.0,0,0,25.9292,S,1,1
436,3,"Ford, Miss. Doolina Margaret ""Daisy""",female,21.0,2,2,34.3750,S,0,0


In [29]:
# from sklearn import svm
# clf = svm.SVC()
# clf.fit(X_train, y_train)

In [59]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [60]:
ct = ColumnTransformer(
    [("imputador", SimpleImputer(), ["Age", "Fare"]),
     ("encoder", OneHotEncoder(), ["Sex", "Embarked"]),
     ("nada", "passthrough", ["Pclass", "SibSp", "Parch", "is_mrs"])]
)

x_prueba = ct.fit_transform(X_train)

In [61]:
len(x_prueba[0])

11

In [64]:
y_train

6      0
576    1
445    1
74     1
670    1
      ..
107    1
271    1
862    1
436    0
103    0
Name: Survived, Length: 595, dtype: int64

In [62]:
clf.fit(x_prueba, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Specifying the columns using strings is only supported for pandas DataFrames

  FitFailedWarning)


ValueError: ignored

In [65]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([("transformador", ct), ("svc", svm.SVC())])

In [66]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('transformador',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('imputador',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0),
                                                  ['Age', 'Fare']),
                                                 ('encoder',
                                                  OneHotEncoder(categories='auto',
   

In [67]:
from sklearn.metrics import accuracy_score
y_pred = pipe.predict(X_test)
accuracy_score(y_test, y_pred)

0.673469387755102

In [68]:
from sklearn.model_selection import GridSearchCV

parametros = {"svc__kernel": ["linear", "rbf"], "svc__C": [1, 10]}

clf = GridSearchCV(pipe, parametros)
clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('transformador',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('imputador',
                                                                         SimpleImputer(add_indicator=False,
                                                                                       copy=True,
                                                                                       fill_value=None,
                                                                                       missing_values=nan,
                                             

In [69]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7959183673469388

In [41]:
df_test = pd.read_csv("test.csv")
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [42]:
df_test["Survived"] = clf.predict(df_test)

/usr/local/lib/python3.7/dist-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


In [43]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0


In [44]:
df_test.loc[:, ["PassengerId", "Survived"]].to_csv("results.csv", index=False)